## CSV dosyasındaki verinin yüklenmesini gerçekleştirme

In [ ]:

from langchain_community.document_loaders.csv_loader import CSVLoader

filepath = "IMDb_Top_1000_Movies_Dataset.csv"
loader = CSVLoader(filepath, encoding="utf-8")

data = loader.load()

print(f"Toplam {len(data)} film yüklendi.")

## Veriyi parçalarına ayırma(Chunking işlemi)
## Her film için Document objesi oluştur

In [ ]:
from langchain_core.documents import Document
import csv
docs = []
with open("IMDB_Top_1000_Movies_Dataset.csv", 'r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    
    for row in csv_reader:
        # Runtime'ı sayısal değere çevir (örn: "142 min" -> 142)
        runtime_str = row.get('Movie_Runtime', '0')
        runtime_minutes = int(''.join(filter(str.isdigit, runtime_str))) if runtime_str else 0
        
        # Film bilgilerini yapılandırılmış bir şekilde hazırla
        content = f"""
Film Adı: {row.get('Movie_Name', 'N/A')}
IMDB Puanı: {row.get('Movie_Rating', 'N/A')}
Yıl: {row.get('Movie_Year', 'N/A')}
Süre: {row.get('Movie_Runtime', 'N/A')} ({runtime_minutes} dakika)
Tür: {row.get('Movie_Genre', 'N/A')}
Sertifika: {row.get('Movie_Certificate', 'N/A')}
Oyuncular: {row.get('Movie_Cast', 'N/A')}
Açıklama: {row.get('Movie_Description', 'N/A')}
"""
        
        # Metadata'ya sayısal ve filtrelenebilir alanları ekle
        metadata = {
            'movie_name': row.get('Movie_Name', ''),
            'rating': row.get('Movie_Rating', '0'),
            'year': row.get('Movie_Year', '0'),
            'runtime_minutes': runtime_minutes,
            'runtime_text': row.get('Movie_Runtime', ''),
            'genre': row.get('Movie_Genre', ''),
            'certificate': row.get('Movie_Certificate', ''),
            'cast': row.get('Movie_Cast', ''),
            'source': 'IMDB_Top_1000'
        }
        
        docs.append(Document(page_content=content, metadata=metadata))

## Google Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv


In [ ]:
load_dotenv()  # Load environment variables from .env file
import os
api_key = os.getenv("GOOGLE_API_KEY")
if api_key is None:
    print("HATA: API anahtarı .env dosyasından okunamadı!")
else:
    print(f"Anahtar başarıyla yüklendi. İlk 4 karakter: {api_key[:4]}")

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", task_batch_size=100)
vector = embeddings.embed_query("hello, world!")
vector[:5]

## ChromaDB üzerine kayıt işlemi

In [ ]:
from langchain_chroma import Chroma

In [ ]:
vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10 })

In [ ]:
retrieved_docs = retriever.invoke("Movies starring Meryl Streep")     

len(retrieved_docs)

print(retrieved_docs[1].page_content)


## Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4): Daha kesin ve daha tutarlı cevaplar verilir. Model daha tahmin edilebilir hale gelir.
- Orta Değerler (0.5-0.7): Hem mantıklı hem de yaratıcı cevaplar verilir.
- Yüksek Değerler (0.7-1): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # Gemini 2.5 model
    temperature=0.3,  # Gemini 3.0+ defaults to 1.0
    max_tokens=500,
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain


In [ ]:
system_prompt = (
    "You are a helpful movie recommendation assistant. Use the following movie database context to answer the user's question."
    "Only recommend movies from the provided context"
    "Mention IMDB ratings and year where relevant."
    "If you don't find suitable movies in the context, say so honestly."
    "\n\n"
    "{context}"
)



In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)


## Soru-Cevap Zinciri Oluşturma (LLM + PROMPT)


In [ ]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)

## RAG Zinciri Oluşturma (RAG + LLM)


In [ ]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

## Kullanıcı sorgusunu çalıştırma

In [ ]:
response = rag_chain.invoke({"input": "Movies over 2 hours long"})

print(response["answer"])